In [1]:
import numpy as np
import sklearn.datasets as sk_dataset
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import BayesianRidge
from scipy.io import loadmat

import pymc3 as pm

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def standardize(x):
    std = np.maximum(np.std(x, axis=0), 1/np.sqrt(len(x)))
    mean = np.mean(x, axis=0)
    return (x - mean) / std

def relu(x):
    return np.maximum(0, x)

def one_hot_encoding(label, n_class):
    y = np.zeros([len(label), n_class])
    for i in range(len(label)):
        y[i, label[i]] = 1
    return y

In [3]:
n_node = 10
n_iter = 1000
lam = 1 # regularization parameter, lambda
w_range = [-1, 1] # range of random weights
b_range = [0, 1] # range of random biases
alpha_1 = 10**(-5) # Gamma distribution parameter
alpha_2 = 10**(-5)
alpha_3 = 10**(-5)
alpha_4 = 10**(-5)

dataset = loadmat('coil20.mat')
label = np.array([dataset['Y'][i][0] - 1 for i in range(len(dataset['Y']))])
data = dataset['X']
n_class = 20

# train-test-split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=42)
# kf = KFold(10, True, 1)
val_acc = []
max_index = -1

X_train = standardize(X_train)
n_sample, n_feature = np.shape(X_train)
y = one_hot_encoding(y_train, n_class)

weights = (w_range[1] - w_range[0]) * np.random.random([n_feature, n_node]) + w_range[0]
bias = (b_range[1] - b_range[0]) * np.random.random([1, n_node]) + b_range[0]

### 1) Initialization
a) Compute $\mathbf{D}$ where $\mathbf{D}=\mathbf{[H,X]}$ <br>

In [4]:
h = relu(np.dot(X_train, weights) + np.dot(np.ones([n_sample, 1]), bias))
d = np.concatenate([h, X_train], axis=1)
# d = np.concatenate([d, np.ones_like(d[:, 0:1])], axis=1) # concat column of 1s

b) Compute $\mathbf{D}^T\mathbf{y}, \mathbf{D}^T\mathbf{D}$, and its eigenvalues $\lambda^0_1,\dots,\lambda^0_B$ <br>

In [5]:
dT_y = np.dot(d.T, y)
dT_d = np.dot(d.T, d)
eigen_val = np.linalg.eigvalsh(dT_d)
eigen_val

array([-3.36940655e-12, -2.16035314e-12, -1.93288977e-12, ...,
        2.56590575e+05,  3.04856976e+05,  1.16809052e+06])

c) Initialize $\sigma^2$ and $\gamma$ to default values <br>
Evidence approximation (MAP estimation on the posterior of the hyper-parameters):
$$p(\gamma)=\text{Gamma}(\gamma \mid \alpha_1, \alpha_2)$$
$$p(\sigma^2)=\text{Gamma}(\sigma^{-2} \mid \alpha_3, \alpha_4)$$
$$ \sigma_*^2, \gamma_*^2 = \arg\max \left\{ \int_{\mathbf{R}^B} p(\mathbf{y} \mid \mathbf{X}, \mathbf{\beta}, \sigma^2)p(\mathbf{\beta} \mid \gamma) p(\gamma)p(\sigma^2)\,d\beta \right\}$$

In [6]:
# Evidence approximation
gamma = pm.Model()
with gamma:
    prec = pm.Gamma('prec', alpha=10**(-5), beta=10**(-5))
    var = pm.Gamma('var', alpha=10**(-5), beta=10**(-5))
    beta = pm.Normal('beta', mu=0, tau=prec, shape=(n_feature + n_node, n_class))
    y_obs = pm.Normal('y_obs', mu=pm.math.dot(d, beta), tau=var, observed=y)
    
estimate =  pm.find_MAP(model=gamma)

In [7]:
prec, var, beta = estimate['prec'].item(0), estimate['var'].item(0), estimate['beta']
print('prec: ', prec)
print('var: ', var)

prec:  1745.1822564064164
var:  11416.765119620248


### 2) Posterior update
a) Computer posterior mean $\mathbf{m}$ as in $$\mathbf{m}=\frac{1}{\sigma^2}\Sigma\mathbf{D}^T\mathbf{y}$$ <br>
b) Computer posterior covariance $\Sigma$ as in $$\Sigma^{-1}=\gamma\mathbf{I}+\frac{1}{\sigma^2}\mathbf{D}^T\mathbf{D}$$

In [8]:
covar = np.linalg.inv(prec * np.identity(dT_d.shape[1]) + dT_d / var)
mean = np.dot(covar, dT_y) / var

### 3) Hyper-parameters update
a) Compute updated eigenvalues $$\lambda_i=\frac{1}{\sigma^2}\lambda^0_i,\qquad i=1,\dots,B$$ <br>

In [9]:
lam = eigen_val / var
lam

array([-2.95127956e-16, -1.89226380e-16, -1.69302754e-16, ...,
        2.24748930e+01,  2.67025706e+01,  1.02313616e+02])

b) Update $\gamma$ as in $$\gamma=\frac{\delta + 2\alpha_1}{\|\mathbf{m}\|^2_2 + 2\alpha_2}$$
c) Update $\sigma^2$ as in $$\sigma^2=\frac{\|\mathbf{y}-\mathbf{D}\beta\|^2_2+\alpha_4}{N-\delta+2\alpha_3}$$
where $\delta$ is defined as $$\delta=\sum^B_{i=1}\frac{\lambda_i}{\gamma+\lambda_i}$$

In [10]:
delta = np.sum(np.divide(lam, lam + prec))
prec = (delta + 2 * alpha_1) / (np.sum(np.square(mean)) + 2 * alpha_2)
var = (np.sum(np.square(y - np.dot(d, beta))) + alpha_4) / (n_sample + delta + 2 * alpha_3)


In [11]:
print(var)

0.0017192075077615008


### Predictive Distribution
$$p(y \mid \mathbf{X}, \mathbf{y}, \mathbf{\hat{x}}, \sigma^2, \gamma) = \mathcal{N} (y \mid \mathbf{m}^T \mathbf{d}(\mathbf{\hat{x}}), \phi(\mathbf{\hat{x}})^2)$$
$$\phi(\mathbf{\hat{x}})^2 = \sigma^2 + \mathbf{d^T(\hat{x})} \Sigma \mathbf{d(\hat{x})}

In [ ]:
basic_model = pm.Model()
with basic_model:
    var_phi = var + np.dot(d.T, covar).dot(d)
    pred = pm.Normal('pred', mu=pm.math.dot(mean.T, d), tau=var_phi)